In [1]:
# !pip install sentencepiece

In [2]:
from transformers import pipeline
import pandas as pd

/home/abdelrahman/.local/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-02-28 13:01:52.946226: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-28 13:01:53.217216: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-02-28 13:01:53.217237: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on 

## A Tour of Transformer Applications

We will see some progress bars as hugging face will start to install the model weights into our local machine from the Hugging Face Hub.
The second time you instantiate the pipeline, the library will notice that you’ve already downloaded the weights and will use the cached version instead.

If no frame work is specified the hugging face will install the original framework used with the model, other wise you can use the framework parameter to specify the framework you need to be used, and this is good because it maybe install the two versions of the model with pytorch or tensorflow.

In [3]:
text = """I have paid out the Immigration Health Surcharge with this reference number "IHS109092150PA01", and the IHS sent me an email 'The transaction reference for your payment of USD 3,950.41 is IHS109092150PA01|2023-01-26T11:05:00. On completion of your IHS application you will also be emailed a copy of your IHS reference number, which you will need for your immigration application.'

I have attached a screenshot of the payment.


But the Immigration health surcharge sends me on Feb 7, 2023, top-up payment is needed and the email format is quoted below:

'Name: MR Abdelrahman Hamdy Rezk

IHS reference number: IHS017333073

A change has been made to the details you provided. You need to pay an additional payment of 3964.2 (USD) by 14 February 2023.'

I have attached a screenshot of the email.


So, you have asked me to pay on this reference number: IHS017333073 which has also been on my application, but I have paid out and you have sent me an email with the payment with this reference number: IHS109092150. So please try to solve this problem because they asked me to pay by 14 February 2023, and I already paid, and you have directed me to the next step of my application with another reference number, and I have no option to add the one you have sent me after the IHS payment, so you have to change my payment number of IHS to the IHS number mentioned in my application and you have asked me to pay for it.

I have attached all screenshots required.

"""

# Text Classification

Most of the NLP applications start to know your customer opinion about your services or anything, know the people's sentiments about anything is very helpful, and it takes no time to know the sentiment of some text.
This can be done in just few lines of code using Huggingface transformers, but we will go deeper o fine-tune these models in chapter 2.

Huggingface Transformers allow you to use its apis at various level of abstraction as you need:
- Direct use of the model (As we use the pipeline for the task directly)
- Build your own model
- load the model in your perfared framework
- Fine tune the model

The pipelines is abstract away all the steps needed to convert raw text into a set of predictions from a fine-tuned model.

pipelines is used by provide the nlp task you need to done on the provided text.

In [4]:
classifier = pipeline("text-classification")
# We can use list of texts, and it will also return a list of predictions, and use DataFrame to be more visible
outputs = classifier(text)
pd.DataFrame(outputs)

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


,label,score
0,NEGATIVE,0.997495


## Named Entity Recognition

Now we have known that its a negative comment from one of our customer, and we need to know what is the complains is about, we need to know much more information instead of reading this long complains, we need to know this compain is about a particular item or service, this known as NER, which extract the objects and classify into some of the categories like its about place or product or services and so on.

We can see how the model extracts the objects from the provided text, as well as give it an entity, and aggregation_strategy simple is to use the categories defined by the model.

The scores tell us how confident the model was about the entities it identified

In [5]:
# Same we use pipeline but we have point to another nlp task which is ner
ner_tagger = pipeline('ner', aggregation_strategy="simple")
outputs = ner_tagger(text)
pd.DataFrame(outputs)

No model was supplied, defaulted to dbmdz/bert-large-cased-finetuned-conll03-english and revision f2482bf (https://huggingface.co/dbmdz/bert-large-cased-finetuned-conll03-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


,entity_group,score,word,start,end
0,ORG,0.846629,I,77,78
1,ORG,0.996680,IHS,104,107
2,ORG,0.974536,IHS,248,251
3,ORG,0.978925,IHS,304,307
4,ORG,0.959406,Immigration,437,448
5,PER,0.890806,Abdelrahman Hamdy Rezk,565,587
6,ORG,0.936408,IHS,589,592
7,ORG,0.916784,IHS,1275,1278
8,ORG,0.995584,IHS,1331,1334
9,ORG,0.943982,IHS,1342,1345


## Question Answering
Extracting all the named entities in a text is nice, but sometimes we would like to ask more targeted questions. This is where we can use question answering.

We provide the model with the context(our text) along with questions we need to get answers for this text.

Some questions and answering systems have extracted the answer from the text itself and its called extractive question answering.

In [6]:
q_a = pipeline('question-answering')
question = "What the problem with IHS ?"
output = q_a(question=question, context=text)
pd.DataFrame([output])

No model was supplied, defaulted to distilbert-base-cased-distilled-squad and revision 626af31 (https://huggingface.co/distilbert-base-cased-distilled-squad).
Using a pipeline without specifying a model name and revision in production is not recommended.


,score,start,end,answer
0,0.00029,1216,1264,I have no option to add the one you have sent me


## Summarization
But what if we have a long text, or document that we need to summarize to get a quick overview about that text.

The goal of text summarization is to take a long text as input and generate a short version with all the relevant facts.

In [7]:
summarizer = pipeline('summarization')
output = summarizer(text, max_length=56, clean_up_tokenization_spaces=True)
print(output[0]['summary_text'])

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


 I have paid out the Immigration Health Surcharge with this reference number "IHS109092150PA01", and the IHS sent me on Feb 7, 2023, top-up payment is needed. The transaction reference for your payment of USD 3


## Translation
What if you are not understand English, then you can tralsate the feedback into your lanugage. 

In [8]:
translator = pipeline("translation_en_to_de", model="Helsinki-NLP/opus-mt-en-de")
outputs = translator(text, clean_up_tokenization_spaces=True, min_length=100)
print(outputs[0]['translation_text'])

/home/abdelrahman/.local/lib/python3.8/site-packages/transformers/models/marian/tokenization_marian.py:194: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


Ich habe die Immigration Health Surcharge mit dieser Referenznummer "IHS109092150PA01" bezahlt, und die IHS schickte mir eine E-Mail "Die Transaktion Referenz für Ihre Zahlung von USD 3.950.41 ist IHS109092150PA01 -2023-01-26T11:05:00. Nach Abschluss Ihrer IHS-Anwendung werden Sie auch eine Kopie Ihrer IHS Referenznummer, die Sie für Ihre Einwanderung Anwendung benötigen per E-Mail.' Ich habe einen Screenshot der Zahlung. Aber die Immigration Gesundheitszuschlag schickt mich am 7. Februar 2023, Nachzahlung erforderlich ist und das E-Mail-Format ist unten angegeben: "Name: MR Abdelrahman Hamdy Rezk IHS Referenznummer: IHS017333073 Eine Änderung wurde an die Details, die Sie angegeben. Sie müssen eine zusätzliche Zahlung von 3964.2 (USD) bis 14 Februar 2023 zahlen. "Ich habe einen Screenshot der E-Mail beigefügt. Also, Sie haben mich auf diese Referenznummer bezahlen lassen.


## Text Generation



In [9]:
generator = pipeline("text-generation")
response = "Dear Abdelrahman, I am sorry to hear that your IHS was wrongly assigned to wrong application."
prompt = text + "\n\nCustomer service response:\n" + response
outputs = generator(prompt, max_length=200)
print(outputs[0]['generated_text'])

No model was supplied, defaulted to gpt2 and revision 6c0e608 (https://huggingface.co/gpt2).
Using a pipeline without specifying a model name and revision in production is not recommended.
2023-02-28 13:11:43.131333: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-02-28 13:11:43.131979: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-02-28 13:11:43.132484: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (abdelrahman-ThinkPad-X1-Carbon-3rd): /proc/driver/nvidia/version does not exist
2023-02-28 13:11:43.140682: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instruction

InvalidArgumentError: {{function_node __wrapped__Fill_device_/job:localhost/replica:0/task:0/device:CPU:0}} Dimension -192 must be >= 0 [Op:Fill]